In [ ]:
# Imports
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
import sklearn as sk
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [ ]:
# Read data and construct dataframe

df = pd.read_csv('datasets/tested_molecules.csv')

df.head()

In [ ]:
# Set the columns which we use for predicting data, so it is easy to change if ever necessary.
x_column = 'SMILES'
y_columns = ['PKM2_inhibition', 'ERK2_inhibition']

In [ ]:
# Convert SMILES to fingerprints, so we have usable data training

def smiles_to_fingerprint(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)
        return list(fp)
    else:
        return [0] * 1024
    
df['Fingerprint'] = df['SMILES'].apply(smiles_to_fingerprint)
x_column = 'Fingerprint'

In [ ]:
# Create test and train sets

X = list(df[x_column])
y = df[y_columns].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
# Prediction model

# Define the pipeline
model = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=0)))
])

# Train model
model.fit(X_train, y_train)

# Make prediction using trained model
predictions = model.predict(X_test)
rounded_predictions = (predictions >= 0.5).astype(int)

# Print results
print("Predictions:\n", rounded_predictions)
print("Actual values:\n", y_test.values)